In [1]:
import numpy as np

In [17]:
r_init=10; N=1000

In [18]:
randomTry = np.random.multivariate_normal(np.zeros(r_init), np.diag(np.ones(r_init)), size=N)

In [22]:
aThingy = np.apply_along_axis(lambda x: x/np.linalg.norm(x), 0 ,randomTry)

In [54]:
np.atleast_2d(aThingy[:,1]).transpose().shape

(1000, 1)

In [1]:
def completion_with_rank_estimation_weighted(Y, W, weightMatrix=None, r_init=40, 
                                        tolerance=1e-04, min_iter=10,max_iter=1000, mu=1e-10):
    
    def P_Omega(Y, D): ## Projects data onto set of observed entries,
                   ## Where D_ij=0 means observed, and D_ij=1 mean missing
    
        return(Y*(1-D))
    
  ### Do NOT root the weight matrix!! 
  
    if(weightMatrix is None):
    
      weightMatrix = np.ones(Y.shape)
    
  
  
    def shrink_operator(x, mu): ## A helper function, the shrinkage operator
    
        if (x > 1*mu):
      
            return(x-mu)
      
        elif abs(x)< mu:
      
            return(0)
      
        else:
      
            return(x+mu)
  
    N = Y.shape[0]
  
    Time = Y.shape[1]
  
    Us = np.random.multivariate_normal(np.zeros(r_init), np.diag(np.ones(r_init)), size=N)
    
    Us = np.apply_along_axis(lambda x: x/np.linalg.norm(x), 0 ,Us)
  
    sigma_vec = np.random.normal(loc=0.0, scale=1.0, size=r_init)
  
    Vs = np.random.multivariate_normal(np.zeros(r_init), np.diag(np.ones(r_init)), size=Time)
  
    Vs = np.apply_along_axis(lambda x: x/np.linalg.norm(x), 0 ,Vs)
  
    r = r_init
  
    L_k = P_Omega(Y, 
                  W)
  
    iterating = True
  
    iter_number = 1
   
    while iterating:
    
        L_k_r = L_k
    
        if iter_number==1:
      
          parts_to_update = range(len(sigma_vec))
      
        else:
      
          parts_to_update = np.where(sigma_vec > 0)[0]
      
    
        for r_number in parts_to_update: ## Note: weights don't influence update of u,v (because we normalize)
      
      ## Two algorithms should be identical when matrix is just 1's (unless initialized differently)
      
            Us[:, r_number] = np.apply_along_axis(lambda x: 1/np.sum(x), 1, weightMatrix)*np.matmul((weightMatrix*L_k_r), Vs[:, r_number])
      
            Us[:, r_number] = Us[:, r_number]/np.linalg.norm(Us[:, r_number])
      
            Vs[:, r_number] = np.apply_along_axis(lambda x: 1/np.sum(x), 0, weightMatrix)*np.matmul((weightMatrix*L_k_r).transpose(), 
                                                                                              Us[:, r_number])
      
            Vs[:, r_number] = Vs[:, r_number]/np.linalg.norm(Vs[:, r_number])
      
            mult_thing = np.matmul(np.atleast_2d(Us[:, r_number]).transpose(), np.atleast_2d(Vs[:, r_number]))

            special_number = np.sum(weightMatrix*(mult_thing**2))
      
      # mu/special_number
      
            sigma_vec[r_number] = max(0, shrink_operator(x=np.sum(mult_thing * L_k_r*weightMatrix)/special_number, 
                         mu=mu))
      
    #  print(mean(sigma_vec))

            L_k_r = L_k_r - sigma_vec[r_number] *  mult_thing
      
        L_k_plus_1 = L_k
    
        Z = L_k-L_k_r
    
        L_k_plus_1[W!=0] = Z[W!=0]

        first_num = (np.linalg.norm(P_Omega(weightMatrix*(L_k_plus_1-Z), W), 'fro')
                /np.linalg.norm(P_Omega(weightMatrix, W), 'fro'))
    
    
        second_num = (np.linalg.norm(weightMatrix*(L_k_plus_1-L_k), 'fro')/np.linalg.norm(weightMatrix, 'fro'))
    
        condition1 = (first_num < tolerance) | (second_num  < tolerance)
    
        condition2 = (iter_number >= max_iter)
        
        if condition1 | condition2:
      
            L_k_final = Z
      
            iterating = False
      
            break
      
        else:
      
            L_k = L_k_plus_1
      
            iter_number = iter_number+1
      
            continue
                    
    ## Is updating changing much? 

    return(Z)

  

  
